## data preparation

In [1]:
import os
import shutil
import glob
from tqdm import tqdm

subject ID:
xxx

image number:
xxx

gender:
0 - male
1 - famale

glasses:
0 - no
1 - yes

eye state:
0 - close
1 - open

reflections:
0 - none
1 - low
2 - high

lighting conditions/image quality:
0 - bad
1 - good

sensor type:
01 - RealSense SR300 640x480
02 - IDS Imaging, 1280x1024
03 - Aptina Imagin 752x480

example:
s001_00123_0_0_0_0_0_01.png






In [2]:
raw_dir = r"\\Mac\Home\Downloads\mrlEyes_2018_01"

In [4]:
for dirpath,dirname,filename in os.walk(raw_dir):
    for i in tqdm(([f for f in filename if f.endswith(".png")])):
        if i.split('_')[4]=="0":
            shutil.copy(src=dirpath+"/"+i,dst=r"\\Mac\Home\Downloads\mrlEyes_2018_01\prepared_data\closeEyes")
        elif i.split("_")[4]=="1":
            shutil.copy(src=dirpath+"/"+i,dst=r"\\Mac\Home\Downloads\mrlEyes_2018_01\prepared_data\openEyes")

### Model training

In [5]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator #data augmentation

In [6]:
batchsize = 8

In [7]:
train_data_generator=ImageDataGenerator(rotation_range=0.2,shear_range=0.2,rescale=1/255,validation_split=.2)

In [8]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [9]:
train_data = train_data_generator.flow_from_directory(r"\\Mac\Home\Downloads\mrlEyes_2018_01\prepared_data\train",target_size=(80,80),batch_size=8,class_mode="categorical",subset="training")
validation_data = train_data_generator.flow_from_directory(r"\\Mac\Home\Downloads\mrlEyes_2018_01\prepared_data\train",target_size=(80,80),batch_size=8,class_mode="categorical",subset="validation")

Found 33536 images belonging to 2 classes.
Found 8382 images belonging to 2 classes.


In [10]:
test_data_generator = ImageDataGenerator(rescale=1/255)
test_data =  test_data_generator.flow_from_directory(r"\\Mac\Home\Downloads\mrlEyes_2018_01\prepared_data\test",target_size=(80,80),batch_size=8,class_mode="categorical")


Found 2152 images belonging to 2 classes.


In [11]:
base_model = InceptionV3(include_top=False, weights = "imagenet", input_tensor=Input(shape=(80,80,3)))

In [12]:
head_model = base_model.output

In [13]:
#base_model.summary()

In [14]:
head_model = Flatten()(head_model)
head_model = Dense(64,activation="relu")(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(2,activation="softmax")(head_model)

In [15]:
model = Model(inputs = base_model.input,outputs = head_model)

In [16]:
for layer in base_model.layers:
    layer.trainable= False

In [17]:
# model.summary()

In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [19]:
checkpoint = ModelCheckpoint(r"\\Mac\Home\Downloads\mrlEyes_2018_01\models\model.h5",monitor = "val_loss", save_best_only=True,verbose =3)
earlystop =EarlyStopping(monitor ="val_loss", patience = 7, verbose =3, restore_best_weights = True)
learning_rate = ReduceLROnPlateau(monitor = "val_loss", patience = 3 ,verbose =3)
callbacks =  [checkpoint,earlystop,learning_rate]

In [20]:
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics = ["accuracy"])
model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,validation_data=validation_data,
                   validation_steps = validation_data.samples//batchsize,
                   callbacks =callbacks,
                   epochs =5)

C:\Users\MUFSEE~1\AppData\Local\Temp/ipykernel_8612/1352404119.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,validation_data=validation_data,


Epoch 1/5
4192/4192 [==============================] - ETA: 0s - loss: 0.0640 - accuracy: 0.9783 ETA
Epoch 00001: val_loss improved from inf to 1.00760, saving model to \\Mac\Home\Downloads\mrlEyes_2018_01\models\model.h5
4192/4192 [==============================] - 560s 133ms/step - loss: 0.0640 - accuracy: 0.9783 - val_loss: 1.0076 - val_accuracy: 0.6391 - lr: 0.0010
Epoch 2/5
4192/4192 [==============================] - ETA: 0s - loss: 0.0325 - accuracy: 0.9899
Epoch 00002: val_loss improved from 1.00760 to 0.29908, saving model to \\Mac\Home\Downloads\mrlEyes_2018_01\models\model.h5
4192/4192 [==============================] - 562s 134ms/step - loss: 0.0325 - accuracy: 0.9899 - val_loss: 0.2991 - val_accuracy: 0.8226 - lr: 0.0010
Epoch 3/5
4192/4192 [==============================] - ETA: 0s - loss: 0.0253 - accuracy: 0.9916
Epoch 00003: val_loss did not improve from 0.29908
4192/4192 [==============================] - 558s 133ms/step - loss: 0.0253 - accuracy: 0.9916 - val_loss: 0

### model evaluation

In [24]:

scores = model.evaluate_generator(train_data)
print("%s%s: %.2f%%" % ("evaluate_generator ",model.metrics_names[1], scores[1]*100))

C:\Users\MUFSEE~1\AppData\Local\Temp/ipykernel_8612/1348140789.py:4: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(train_data)


evaluate_generator accuracy: 99.75%


In [25]:
scores = model.evaluate_generator(validation_data)
print("%s%s: %.2f%%" % ("evaluate_generator ",model.metrics_names[1], scores[1]*100))

C:\Users\MUFSEE~1\AppData\Local\Temp/ipykernel_8612/1372518863.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(validation_data)


evaluate_generator accuracy: 74.23%


In [26]:
scores = model.evaluate_generator(test_data)
print("%s%s: %.2f%%" % ("evaluate_generator ",model.metrics_names[1], scores[1]*100))

C:\Users\MUFSEE~1\AppData\Local\Temp/ipykernel_8612/961241508.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(test_data)


evaluate_generator accuracy: 90.06%
